In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
from google.colab.patches import cv2_imshow
import sys

In [ ]:
from IPython.display import display, Javascript,HTML
from google.colab.output import eval_js
from base64 import b64decode
 
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
       
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";
 
      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);
 
      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';
 
      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
     
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
 
      video.srcObject = stream;
      video.muted = true;
 
      await video.play();
 
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
 
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
 
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
       
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();
 
      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))

In [ ]:
video_path = "test_video.mp4"
record_video(video_path)

**Create folder for frames**

In [ ]:
import os
os.makedirs('/content/drive/MyDrive/Myphotos/Datasets')

**Counting the no. of frames in a video**

In [ ]:
cap=cv2.VideoCapture('/content/test_video.mp4')
while(cap.isOpened()):
  ret,frame=cap.read()
  if ret:
    if cv2.waitKey(10) == 27:                     # exit if Escape is hit
      break
  else:
    break
  count += 1
print(count)

606


pip install python-ffmpeg

This python-ffmpeg gives error. install ffmpeg-python

In [ ]:
!pip install ffmpeg-python


In [ ]:
import ffmpeg

mpdecimate and setpts are filters. vf is video filter.

-an is removing audio. mpdecimate only operates on video media file.

mpdecimate - it analyzes all the frames in the video and if it finds a sequence where it doesn't looks like anything is changing. It will remove all of them. So you get one piece of information that is useful. 

if you want it by itself you will get a section where nothing happens because the bunch of frames is being removed.


setpts=it updates the presentation timestamps on the frames that are left. Nice clean playback of the frames left after the frozen frames were dropped.

In [ ]:
#ffmpeg -i test1.mp4 -vf mpdecimate,setpts=N/FRAME_RATE/TB -an test1_decimated.mp4

In [ ]:
import ffmpeg
(
    ffmpeg
    .input('/content/test_video.mp4')
    .filter_('mpdecimate')
    .filter_('setpts','N/FRAME_RATE/TB')
    .output('test_decimated.mp4')
    .run()
)

(None, None)

**Adding frames to dataset**

In [ ]:
cap=cv2.VideoCapture('/content/drive/MyDrive/Myphotos/Datasets/test_decimated.mp4')
count=0
while(cap.isOpened()):
  ret,frame=cap.read()
  if ret:
    cv2.imwrite( '/content/drive/MyDrive/Myphotos/Datasets'+ "/%#05d.jpg" %count, frame)
    if cv2.waitKey(10) == 27:                     # exit if Escape is hit
      break
  else:
    break
  count += 1

In [ ]:
modelFile = "/content/drive/MyDrive/Myphotos/model/res10_300x300_ssd_iter_140000_fp16 (1).caffemodel"
configFile = "/content/drive/MyDrive/Myphotos/model/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [ ]:
cap = cv2.VideoCapture('/content/test_video.mp4')
count=0
ret=True

vs = VideoStream(src=0).start()
time.sleep(2)

fps = FPS().start()
while ret:
  count+=1
  print(count)
  ret, frame=cap.read()
  try:
    frame = cv2.resize(frame , (320,240))
    frame = imutils.resize(frame, width=600)
  
  except:
    continue

  (h, w) = frame.shape[:2]
  
  imageBlob = cv2.dnn.blobFromImage(
      cv2.resize(frame, (300, 300)), 1.0, (300, 300),
      (104.0, 177.0, 123.0), swapRB=False, crop=False)
  
  net.setInput(imageBlob)
  detections = net.forward()
  print(detections)
  
  for i in range(0, detections.shape[2]):
    
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.95:
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")
      
      face = frame[startY:endY, startX:endX]
      (fH, fW) = face.shape[:2]
      
      if fW < 20 or fH < 20:
        continue
      cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
    else:
      print('*')
      continue
  fps.update()

  cv2_imshow(frame)
  if(cv2.waitKey(1) & 0xFF==27):
    break
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cv2.destroyAllWindows()
vs.stop()